In [1]:
from utils import load_model_and_files, get_model_details, reconstruct_word, merge_results, preprocess_text, preprocess_stopwords, preprocess_lemmatization

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, MarianMTModel, MarianTokenizer

### carico i modelli

In [3]:
# 1. Recupera gli ID dei modelli e i relativi file dal file .env
(model_1_id, model_1_files), (model_2_id, model_2_files), (model_3_id, model_3_files), (model_4_id, model_4_files) = get_model_details()

# 2. Carica le pipeline per i modelli
print(model_1_id)
model_1_pipeline = load_model_and_files(model_1_id, model_1_files)

print(model_2_id)
model_2_pipeline = load_model_and_files(model_2_id, model_2_files)

print(model_3_id)
model_3_pipeline = load_model_and_files(model_3_id, model_3_files)

print(model_4_id)
model_4_pipeline = load_model_and_files(model_4_id, model_4_files)

osiria/bert-italian-uncased-ner


Device set to use cpu


IVN-RIN/MedPsyNIT


Device set to use cpu


SamLowe/roberta-base-go_emotions


Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at SamLowe/roberta-base-go_emotions and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


zaneas/Traduttore_IT_EN_2
Errore nel caricamento del modello zaneas/Traduttore_IT_EN_2: Unrecognized configuration class <class 'transformers.models.marian.configuration_marian.MarianConfig'> for this kind of AutoModel: AutoModelForTokenClassification.
Model type should be one of AlbertConfig, BertConfig, BigBirdConfig, BioGptConfig, BloomConfig, BrosConfig, CamembertConfig, CanineConfig, ConvBertConfig, Data2VecTextConfig, DebertaConfig, DebertaV2Config, DistilBertConfig, ElectraConfig, ErnieConfig, ErnieMConfig, EsmConfig, FalconConfig, FlaubertConfig, FNetConfig, FunnelConfig, GemmaConfig, Gemma2Config, GlmConfig, GPT2Config, GPT2Config, GPTBigCodeConfig, GPTNeoConfig, GPTNeoXConfig, IBertConfig, LayoutLMConfig, LayoutLMv2Config, LayoutLMv3Config, LiltConfig, LlamaConfig, LongformerConfig, LukeConfig, MarkupLMConfig, MegaConfig, MegatronBertConfig, MistralConfig, MixtralConfig, MobileBertConfig, MPNetConfig, MptConfig, MraConfig, MT5Config, NemotronConfig, NezhaConfig, Nystromformer

In [4]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_4_id)
tokenizer = AutoTokenizer.from_pretrained(model_4_id)

c:\Users\giuly\Documents\GitHub\data_semantics\.venv\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


### faccio una prova utilizzando direttamente i modelli sul testo crudo

In [ ]:
# 3. Definisci il testo su cui eseguire l'analisi
text = "Vorrei sapere se Giulia che vive Milano c'è qualche (seria)speranza anche per chi - dopo anni di problemi seri (endometriosi e fibromi uterino) - sta raggiungendo i 50 anni. (Sono consapevole di essere anzianotta, ma la speranza c'è sempre). Grazie"

# 4. Esegui l'analisi sui modelli solo se le pipeline sono caricate correttamente
result_1 = []
if model_1_pipeline:
    result_1 = model_1_pipeline(text)
else:
    print(f"Errore nel caricare la pipeline per il modello 1: {model_1_id}")

result_2 = []
if model_2_pipeline:
    result_2 = model_2_pipeline(text)
else:
    print(f"Errore nel caricare la pipeline per il modello 2: {model_2_id}")

# 5. Ricostruisci i risultati dai sub-token in parole complete per entrambi i modelli
reconstructed_model_1_results = reconstruct_word(result_1) if result_1 else []
reconstructed_model_2_results = reconstruct_word(result_2) if result_2 else []

# 6. Unisci i risultati dei due modelli, con la preferenza per il modello 2
final_results = merge_results(reconstructed_model_1_results, reconstructed_model_2_results)

# 7. Stampa i risultati finali in modo leggibile
print(text)
print("\nRisultati finali dopo unione dei modelli:")
for result in final_results:
    print(f"Parola: {result['word']}, Entità: {result['entity']}, Score: {result['score']:.4f}")
    print("--------------------------------")

Vorrei sapere se Giulia che vive Milano c'è qualche (seria)speranza anche per chi - dopo anni di problemi seri (endometriosi e fibromi uterino) - sta raggiungendo i 50 anni. (Sono consapevole di essere anzianotta, ma la speranza c'è sempre). Grazie

Risultati finali dopo unione dei modelli:
Parola: giulia, Entità: PER, Score: 0.9279
--------------------------------
Parola: milano, Entità: LOC, Score: 0.7912
--------------------------------
Parola: endometriosi, Entità: DIAGNOSI E COMORBIDITA (B), Score: 0.7861
--------------------------------
Parola: fibromi, Entità: DIAGNOSI E COMORBIDITA (B), Score: 0.8488
--------------------------------
Parola: uterino, Entità: DIAGNOSI E COMORBIDITA (I), Score: 0.7146
--------------------------------


### faccio una prova puleno il testo

In [8]:
# 3. Definisci il testo su cui eseguire l'analisi
text = "Vorrei sapere se Giulia che vive Milano c'è qualche (seria)speranza anche per chi - dopo anni di problemi seri (endometriosi e fibromi uterino) - sta raggiungendo i 50 anni. (Sono consapevole di essere anzianotta, ma la speranza c'è sempre). Grazie"

text_processed = preprocess_text(text)
text_processed = preprocess_stopwords(text_processed)
text_processed = preprocess_lemmatization(text_processed)

# 4. Esegui l'analisi sui modelli solo se le pipeline sono caricate correttamente
result_1 = []
if model_1_pipeline:
    result_1 = model_1_pipeline(text_processed)
else:
    print(f"Errore nel caricare la pipeline per il modello 1: {model_1_id}")

result_2 = []
if model_2_pipeline:
    result_2 = model_2_pipeline(text_processed)
else:
    print(f"Errore nel caricare la pipeline per il modello 2: {model_2_id}")

# 5. Ricostruisci i risultati dai sub-token in parole complete per entrambi i modelli
reconstructed_model_1_results = reconstruct_word(result_1) if result_1 else []
reconstructed_model_2_results = reconstruct_word(result_2) if result_2 else []

# 6. Unisci i risultati dei due modelli, con la preferenza per il modello 2
final_results = merge_results(reconstructed_model_1_results, reconstructed_model_2_results)

# 7. Stampa i risultati finali in modo leggibile
print(text)
print(text_processed)
print("\nRisultati finali dopo unione dei modelli:")
for result in final_results:
    print(f"Parola: {result['word']}, Entità: {result['entity']}, Score: {result['score']:.4f}")
    print("--------------------------------")

Vorrei sapere se Giulia che vive Milano c'è qualche (seria)speranza anche per chi - dopo anni di problemi seri (endometriosi e fibromi uterino) - sta raggiungendo i 50 anni. (Sono consapevole di essere anzianotta, ma la speranza c'è sempre). Grazie
volere sapere Giulia vivere Milano c ' serio speranza problema serio endometrioso fibrome uterino raggiungere il 50 . consapevole anzianotta , speranza c ' .

Risultati finali dopo unione dei modelli:
Parola: giulia, Entità: MISC, Score: 0.3940
--------------------------------
Parola: milano, Entità: MISC, Score: 0.5187
--------------------------------
Parola: speranza, Entità: MISC, Score: 0.3922
--------------------------------
Parola: endometrioso, Entità: DIAGNOSI E COMORBIDITA (B), Score: 0.4980
--------------------------------
Parola: fime, Entità: DIAGNOSI E COMORBIDITA (I), Score: 0.5342
--------------------------------
Parola: uterino, Entità: DIAGNOSI E COMORBIDITA (I), Score: 0.6979
--------------------------------


In [8]:
from transformers import pipeline

classifier = pipeline(task="text-classification", model=model_3_id, top_k=None)

sentences = ["I would like to know if Giulia who lives Milan there is some (serious) hope also for those who - after years of serious problems (endometrosis and uterine fibroids) - are reaching 50 years. (I am aware that I am elderly, but there is always hope)"]

model_outputs = classifier(sentences)
print(model_outputs[0])
# produces a list of dicts for each of the labels


Device set to use cpu


[{'label': 'desire', 'score': 0.44079965353012085}, {'label': 'curiosity', 'score': 0.31969621777534485}, {'label': 'optimism', 'score': 0.2585451006889343}, {'label': 'neutral', 'score': 0.0955871194601059}, {'label': 'confusion', 'score': 0.02322000078856945}, {'label': 'caring', 'score': 0.0206000916659832}, {'label': 'approval', 'score': 0.019204890355467796}, {'label': 'disappointment', 'score': 0.011250891722738743}, {'label': 'sadness', 'score': 0.009548953734338284}, {'label': 'admiration', 'score': 0.006543469615280628}, {'label': 'love', 'score': 0.006385591346770525}, {'label': 'excitement', 'score': 0.006221286486834288}, {'label': 'disapproval', 'score': 0.005997466389089823}, {'label': 'realization', 'score': 0.005887140985578299}, {'label': 'annoyance', 'score': 0.005676315166056156}, {'label': 'surprise', 'score': 0.005030387546867132}, {'label': 'fear', 'score': 0.0038803881034255028}, {'label': 'remorse', 'score': 0.0031880855094641447}, {'label': 'nervousness', 'scor

In [7]:
text = "Vorrei sapere se Giulia che vive Milano c'è qualche (seria)speranza anche per chi - dopo anni di problemi seri (endometriosi e fibromi uterino) - sta raggiungendo i 50 anni. (Sono consapevole di essere anzianotta, ma la speranza c'è sempre). Grazie"


def traduci_testo(testo):
    # Carica il modello e il tokenizer per italiano -> inglese
    model_name = "Helsinki-NLP/opus-mt-it-en"
    model = MarianMTModel.from_pretrained(model_name)
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    
    # Tokenizza il testo
    inputs = tokenizer(testo, return_tensors="pt", padding=True, truncation=True)
    
    # Ottieni la traduzione
    translated = model.generate(**inputs)
    
    # Decodifica la traduzione
    tradotto = tokenizer.decode(translated[0], skip_special_tokens=True)
    return tradotto

# Testo da tradurre
testo_in_italiano = text
traduzione = traduci_testo(testo_in_italiano)

# Stampa il risultato
print(f"Testo tradotto: {traduzione}")

c:\Users\giuly\Documents\GitHub\data_semantics\.venv\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Testo tradotto: I would like to know if Giulia who lives Milan there is some (serious) hope also for those who - after years of serious problems (endometrosis and uterine fibroids) - are reaching 50 years. (I am aware that I am elderly, but there is always hope).
